<a href="https://colab.research.google.com/github/Alan0520/Final_Project_0620/blob/main/Final_Project_0620.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
%cd /content/drive/MyDrive/tfrecordtoimages2

/content/drive/MyDrive/tfrecordtoimages2


In [10]:
%tensorflow_version 2.x

In [6]:
!pip install waymo-open-dataset-tf-2-1-0==1.2.0

     |████████████████████████████████| 2.8MB 30.8MB/s 
     |████████████████████████████████| 421.8MB 42kB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 450kB 54.8MB/s 
     |████████████████████████████████| 3.9MB 48.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=5b773cb0c97573a4ac00e3fa45949cfd6b4154c4989ca6d5d0b364daed0c53d1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.5.0 has requirement gast==0.4.0, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorflow-estimator<2.6.0,>=2.5.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have 

In [11]:
import tensorflow as tf
import math
import numpy as np
import itertools
import os
import matplotlib.pyplot as plt
from waymo_open_dataset.utils import frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
tf.compat.v1.enable_eager_execution() 

In [12]:
def bounding_box(camera_labels, i, width, height):
    x_center = frame.camera_labels[camera_labels].labels[i].box.center_x / width
    y_center = frame.camera_labels[camera_labels].labels[i].box.center_y / height
    box_w = frame.camera_labels[camera_labels].labels[i].box.length / width
    box_h = frame.camera_labels[camera_labels].labels[i].box.width / height
    return x_center, y_center, box_w, box_h

In [18]:
path = '/content/drive/MyDrive/validation/'

files = os.listdir(path)
for i in range(len(files)):
  print(i, files[i])

0 1.tfrecord
1 labels
2 images


In [ ]:
for i in range(0, len(files)):
    File = files[i]
    dataset = tf.data.TFRecordDataset(path + File, compression_type='')
    count = 0
    print(len(files), i, File)
    for data in dataset:
        count += 1
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        
        print('.', end = '')
        for camera_labels in range(3):
            # image size
            if camera_labels == 2:
                camera_labels = 3
            width = 1920
            height = 1280
    
            # label_filename
            label_filename = ('./labels/1/' + frame.context.name + '_' + str(count).zfill(6)
                            + '_' + str(camera_labels) + '.txt')
            with open(label_filename, 'w') as text_file:
                for i in range(len(frame.camera_labels[camera_labels].labels)):
                    object_class = frame.camera_labels[camera_labels].labels[i].type
                    if object_class == 1:      # Vehicle
                        x_center, y_center, box_w, box_h = bounding_box(camera_labels, i, width, height)
                        text_file.write(str(0)+' '+str(x_center)+' '+str(y_center)+' '+str(box_w)+' '+str(box_h)+'\n')
                    if object_class == 2: # Pedstrian
                        x_center, y_center, box_w, box_h = bounding_box(camera_labels, i, width, height)
                        text_file.write(str(1)+' '+str(x_center)+' '+str(y_center)+' '+str(box_w)+' '+str(box_h)+'\n')
                    if object_class == 4: # Cyclist
                        x_center, y_center, box_w, box_h = bounding_box(camera_labels, i, width, height)
                        text_file.write(str(2)+' '+str(x_center)+' '+str(y_center)+' '+str(box_w)+' '+str(box_h)+'\n')
    
            sc = 1920/1449;
            width = 1920.0 * sc; height = 1280.0 * sc            
            plt.figure(figsize=(width/72.0, height/72.0), dpi = 72.0)
            plt.axis('off');  plt.grid(False)
            plt.imshow(tf.image.decode_jpeg(frame.images[camera_labels].image), cmap=None)
            
            image_filename = ('./images/1/' + frame.context.name + '_' + str(count).zfill(6)
                                + '_' + str(camera_labels) + '.jpg')
            plt.savefig(image_filename, bbox_inches='tight',pad_inches = 0)
            plt.close()
    print("\n")

3 0 1.tfrecord
................................................................................................................................................................